In [30]:
import numpy as np
import sys, os
sys.path.append(os.pardir + '/deep-learning-from-scratch/') 
from dataset.mnist import load_mnist
from PIL import Image
import pickle 
from common.functions import *
from common.gradient import numerical_gradient
from tqdm import tqdm

In [25]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std= 0.01):
        # 重みの初期化
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['b2'] = np.zeros(output_size)

    # 推論を行う
    # xは画像データ
    def predict(self, x):
        # 変数の定義
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        
        # 各層の演算
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)

        return y

    # 損失関数を求める
    # x:画像データ、t:正解ラベル
    def loss(self, x, t):
        y = self.predict(x)
        loss = cross_entropy_error(y, t)
        return loss
    
    # 精度算出
    def accurancy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)

        accurancy = np.sum(y == t)/float(x.shape[0])
        return accurancy
    
    # 重みパラメータの勾配を求める
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)

        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

        return grads


In [ ]:
(x_train, y_train), (x_test, y_test) = load_mnist(normalize=True, one_hot_label=True)

train_loss_list = []

#　ハイパーパラメータ
iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
lr = 0.01

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

for i in tqdm(range(iters_num)):
    # ミニバッチの取得
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    y_batch = y_train[batch_mask]

    # 勾配の計算
    grad = network.numerical_gradient(x_batch, y_batch)

    # パラメータの更新
    for key in ('W1', 'W2', 'b1', 'b2'):
        network.params[key] -= lr*grad[key]

    # 学習経過の記録
    loss = network.loss(x_batch, y_batch)
    train_loss_list.append(loss)
